In [1]:
# import pandas as pd
# import requests
# from bs4 import BeautifulSoup
# import logging
# from urllib.parse import urljoin
# import nltk
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# from nltk.stem import PorterStemmer
# from textblob import TextBlob
# import csv
# import re 

In [2]:
# class ClawWeb:
#     def __init__(self, url:str):
#         r = requests.get(url)
#         self.soup = BeautifulSoup(r.text, "html.parser")
#         self.link = self.get_link()
#         self.head = self.get_head()
#         self.body = self.get_body()
#         # self.content = self.get_content()
#         self.save = self.get_save()
#         # self.senti = self.sentiment()

#     def get_link(self):
#         issue = self.soup.find_all('a', href = True, attrs = {'class' : 'qa-heading-link lx-stream-post__header-link'})
#         self.list_link = []
#         for i in issue:
#             if "https://www.bbc.com" in i['href']:
#                 self.list_link.append(i['href'])
#             else:
#                 self.list_link.append("https://www.bbc.com"+i['href'])
#         return self.list_link

#     def get_head(self):
#         issue = self.soup.find_all('a', href = True, attrs = {'class' : 'qa-heading-link lx-stream-post__header-link'})
#         list_head = []
#         for i in issue:
#             # print('test', i.string)
#             if i.string == None:
#                 pass
#             else:
#                 if i.string not in list_head:
#                     list_head.append(i.string)
#         return list_head

#     def get_body(self):
#         issue = self.soup.find_all('p', {'class' : 'lx-stream-related-story--summary qa-story-summary'})
#         list_body = []
#         for i in issue:
#             if i.string == None:
#                 pass
#             else:
#                 if i.string not in list_body:
#                     list_body.append(i.string)
#         return list_body

    
#     # def get_content(self):
#     #     issue = self.soup.find_all('p', {'class' : 'ssrcss-1q0x1qg-Paragraph eq5iqo00'})

#     #     list_content = []
#     #     for k in issue:
#     #         print('test', k)
#     #         if k.string == None:
#     #             list_content.append("nan")
#     #         else:
#     #             # print(k.string)
#     #             list_content.append(k.string)
#     #     return list_content

    
#     def sentiment(self, text_en):
#         sentiment_text = text_en.sentiment.polarity
#         if  sentiment_text > 0:
#             return 'positive'
#         elif sentiment_text < 0:
#             return 'negative'
#         else:
#             return 'neutral'

#     def list_to_text(self ,txt):
#         new_head = []
#         for i in txt:
#             # print("i",i)     
#             new_head.append(self.sentiment(TextBlob(i)))
#         return new_head


#     def get_save(self):
#         table = {"head" : self.head, "sentiment" : self.list_to_text(self.head), "link" : self.link}
#         df = pd.DataFrame(data = table)
#         df.to_excel('testweb.xlsx', index=False)
#         return df


In [3]:
import re
import json
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import pandas as pd


In [4]:
class testNME:

    def __init__(self):
        self.data = {}
        self.data['web'] = {}

    
    def remove_unuse_tag(self, bs : BeautifulSoup):
        unuse_tag = ['script', 'style ', 'noscript', 'head', 'footer', 'iframe']
        for tag in unuse_tag:
            for s in bs.select(tag):
                if s != None:
                    s.extract()
        return bs

    def clean_html(self, html : str):
        clean = re.compile('<.*?>')
        clean_text = re.sub(clean, '', html)
        for char in ['\n', '\t', '\r']:
            clean_text = clean_text.replace(char, '')
        clean_text = re.sub(' +', ' ', clean_text)
        return clean_text

    def data_in_link(self, session, url):
        self.list_url = []
        self.list_title = []
        self.list_content = []

        with session.get(url) as response:
            if not response.ok:
                return
            print('test url', url)
            
            domain = urlparse(url).netloc
            bs = BeautifulSoup(response.text, 'html.parser')
            bs = self.remove_unuse_tag(bs)
            # print('test bs inlink', bs)
            
            self.data['web'][url] = {}
            self.data['web'][url]['title'] = bs.find('h1').text

            self.content = ''
            section = bs.find('div', {'data-td-block-uid' : 'tdi_95'})
            # print('test sec', section)
            for p in section.find_all('p'):
                # print('test p', p)
                self.content += f'{self.clean_html(p.text)} ' #p.text
            self.data['web'][url]['content'] = self.content
            # print('test content inlink', self.content)



    def data_domain(self, session, url):
        with session.get(url) as response:
            # print('check re', response)
            if not response.ok:
                return
            domain = urlparse(url).netloc
            bs = BeautifulSoup(response.text, 'html.parser')
            bs = self.remove_unuse_tag(bs)
            # print('test bs domain', bs)

            news_link = []
            for h3 in bs.find_all('h3', {'class' : 'entry-title'}):
                a = h3.find('a')
                news_link.append(a.attrs['href'])
            # print('test news_link domain', news_link)

            n = len(news_link)
            # print('test n news_link domain', n)
            with ThreadPoolExecutor(max_workers=n) as executor:
                with requests.Session() as session:
                    executor.map(self.data_in_link, [session]*n, news_link)
                    executor.shutdown(wait=True)


    def scrap(self):
        pages_list = [f'https://www.nme.com/news/music/page/{i}' for i in range(1, 11)]
        n = len(pages_list)
        with ThreadPoolExecutor(max_workers=n) as executor:
            with requests.Session() as session:
                executor.map(self.data_domain, [session]*n, pages_list)
                executor.shutdown(wait=True)


    def save_to_json(self):
        with open(f'NME.json', 'w', encoding="UTF-8") as file:
            datafile = json.dumps(self.data, indent=4) 
            file.write(datafile)


    # def dataframe_web(self):

    #     data_web = [[self.list_url
    #                     ]]

    #     web_frame = pd.DataFrame(data=data_web,
    #                         columns=['link'])
    #     return web_frame

    # def twitter_to_xlsx(self, data):
    #     data.to_excel("hey_test.xlsx", engine="openpyxl", index=False)


In [5]:
data = testNME()
data

In [6]:
data.scrap()

test url https://www.nme.com/news/music/pooh-shiesty-sentenced-to-five-years-in-prison-over-florida-shooting-3209449
test url https://www.nme.com/news/music/drake-granted-three-year-restraining-order-against-alleged-stalker-3209294
test url https://www.nme.com/news/music/big-thiefs-adrianne-lenker-asks-fans-to-stop-talking-during-support-act-sets-3209515
test urltest url https://www.nme.com/news/music/pooh-shiesty-sentenced-to-five-years-in-prison-over-florida-shooting-3209449
 https://www.nme.com/news/music/the-kid-laroi-returns-with-new-single-and-video-thousand-miles-3209841
test urltest url https://www.nme.com/news/music/epik-high-tablo-says-big-bang-taeyang-favourite-person-to-work-with-3210235
test url https://www.nme.com/news/music/ed-sheeran-shares-lil-baby-remix-of-2step-with-royalties-going-to-ukraine-relief-3210230
test url  https://www.nme.com/news/music/b-i-teases-new-global-album-project-for-2022-3209303
test url https://www.nme.com/news/music/harry-styles-coachella-crowd

In [7]:
# data.dataframe_web()

In [8]:
data.save_to_json()

In [9]:
# hey = data.dataframe_web()
# data.twitter_to_xlsx(hey)